In [ ]:
#default_exp learner

In [ ]:
#export
from detectrino.basics import *
from fastai2.vision.all import *

# Learner
> Core Learner

In [ ]:
#export
# Dsets is only train dataset for now
# TODO: Loading datset twice: Once for getting the len, other by detectron2 internals
# TODO: How to correctly handle max_iter and lr_schedule?
class DetLearner:
    def __init__(self, dset, mcfg, pretrained=True, train_name='train', valid_name='valid'):
        self.dset_train,self.dset_valid = f'{dset}_{train_name}',f'{dset}_{valid_name}'
        self.dset_len = self._dset_len(self.dset_train)
        cfg = get_cfg()
        cfg.merge_from_file(model_zoo.get_config_file(mcfg.mfile))
        self.cfg = cfg = mergedicts(cfg, mcfg.to_cfg())
        cfg.DATASETS.TRAIN,cfg.DATASETS.TEST = [self.dset_train],[]
        self.trainer = DefaultTrainer(cfg)
        cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(mcfg.mfile)
        if pretrained: self.trainer.resume_or_load(False)
        os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
        self.reload = False # Used for reloading predictor
        self._predictor = None
        
    def fit(self, n_epoch, lr, bs=None):
        self.reload = True
        self.update_solver_cfg(n_epoch, lr, bs)
        self.trainer = DefaultTrainer(self.cfg)
        self.trainer.train()
        self.cfg.MODEL.WEIGHTS = str(self.path/'model_final.pth')
        
    def load(self, name):
        self.reload = True
        self.cfg.MODEL.WEIGHTS = str(self.path/name)
        self.trainer.resume_or_load(False)
        
    def _dset_len(self, dset): 
#         return len(self.trainer.data_loader.dataset.dataset._dataset._addr)
        return len(DatasetCatalog.get(dset))

    def update_solver_cfg(self, n_epoch, lr, bs=None):
        cfgs = self.cfg.SOLVER
        bs = bs or cfgs.IMS_PER_BATCH
        max_iter = int(n_epoch*(self.dset_len/bs))
        scale = (max_iter/cfgs.MAX_ITER)
        cfgs.BASE_LR = lr
        cfgs.MAX_ITER = max_iter
        cfgs.IMS_PER_BATCH = bs
        cfgs.STEPS = tuple((np.array(cfgs.STEPS)*scale).astype(int))
        cfgs.WARMUP_ITERS = max(100, int(cfgs.WARMUP_ITERS*scale))
        cfgs.CHECKPOINT_PERIOD = max(200, int(cfgs.CHECKPOINT_PERIOD*scale))

    @property
    def path(self): return Path(self.trainer.cfg.OUTPUT_DIR)

# Predict

In [ ]:
#export
@patch_property
def predictor(self:DetLearner):
    if self._predictor is None or self.reload:
        cfg = self.cfg.clone()
        cfg.DATASETS.TEST = cfg.DATASETS.TRAIN
        self._predictor = DefaultPredictor(cfg)
        self.reload = False
    return self._predictor

In [ ]:
#export
@patch
def predict(self:DetLearner, fn, thresh=.5):
    predictor = self.predictor
    predictor.model.roi_heads.test_score_thresh = thresh
    im = PILImage.create(fn)
    pred = predictor(im.cv2())
    v = Visualizer(im.numpy(), predictor.metadata, scale=.5)
    v = v.draw_instance_predictions(pred['instances'].to('cpu'))
    return PILImage.create(v.get_image())

## Evaluate

In [ ]:
#export
@patch
def evaluate(self:DetLearner, thresh=.5):
    cfg = self.cfg.clone()
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = thresh
    cfg.DATASETS.TEST = [self.dset_valid]
    evaluator = COCOEvaluator(self.dset_valid, self.cfg, False, output_dir='./output/')
    val_loader = build_detection_test_loader(cfg, self.dset_valid)
    inference_on_dataset(self.trainer.model, val_loader, evaluator)

In [ ]:
# TODO: Simple test
# mfile = ModelCatalog.COCO_InstanceSegmentation__mask_rcnn_R_50_FPN_3x
# learn = DetLearner('voc_2007_val', RCNNCfg(mfile, 12, 128))

## Export -

In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_data.core.ipynb.
Converted 02_data.config.ipynb.
Converted 05_model.external.ipynb.
Converted 06_model.config.ipynb.
Converted 08_learner.ipynb.
Converted index.ipynb.
